In [ ]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [ ]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

orig_dat = alldat #Save a copy of original

print(alldat[11].keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [ ]:
#@title Add New Keys
import scipy.io as sio
import requests
import io

rtstruct_url = 'https://github.com/ayeshav/nma_project/blob/master/RTStruct.mat?raw=true'
r = requests.get(rtstruct_url)

temp = sio.loadmat(io.BytesIO(r.content))
RTStruct = temp['RTStruct']
rt = RTStruct[0]['allRt']
rt_trial_indices = RTStruct[0]['trialInd']

alldat=orig_dat #Set alldat to original
nSessions = len(alldat)

for iSession in range(nSessions):
  dat = alldat[iSession]
  relative_contrast = dat['contrast_left'] - dat['contrast_right']
  reaction_time = rt[iSession]
  rt_trial_ind = rt_trial_indices[iSession] -1 # Subtract 1 because MATLAB is 1-indexed, Python is 0-indexed

  nTrials = len(dat['gocue'])

  alldat[iSession]['reaction_time'] = np.empty((nTrials))
  alldat[iSession]['reaction_time'][:] = np.nan
  alldat[iSession]['reaction_time'][rt_trial_ind] = reaction_time

  alldat[iSession]['relative_contrast'] = relative_contrast
  alldat[iSession]['difficulty'] = abs(relative_contrast)

  alldat[iSession]['GO_trial'] = (relative_contrast!=0)
  
  alldat[iSession]['GO_correct'] = np.logical_or(np.logical_and(relative_contrast<0,dat['response']<0),np.logical_and(relative_contrast>0,dat['response']>0))
  alldat[iSession]['GO_incorrect'] = np.logical_or(np.logical_and(relative_contrast<0,dat['response']>0),np.logical_and(relative_contrast>0,dat['response']<0))

  alldat[iSession]['GO_correct_right'] = np.logical_and(alldat[iSession]['GO_correct'],relative_contrast<0)
  alldat[iSession]['GO_correct_left'] = np.logical_and(alldat[iSession]['GO_correct'],relative_contrast>0)

  alldat[iSession]['GO_incorrect_right'] = np.logical_and(alldat[iSession]['GO_incorrect'],relative_contrast<0)
  alldat[iSession]['GO_incorrect_left'] = np.logical_and(alldat[iSession]['GO_incorrect'],relative_contrast>0)

